# 3.1 Calculation of probabilistic results (2030)

## Importing libraries

In [ ]:
from brightway2 import *
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rankdata
import string
%matplotlib inline
import pickle


## Projects setting

In [ ]:
projects.set_current('pLCA_RegAC_2030')

## ImpactWorld+ import

In [ ]:
# Import the specified BW2Package for the LCIA method IMPACTWorld+ file from the given path or import it to the same file location as this notebook
BW2Package.import_file("Brightway_IW_damage_1_46_and_midpoint_1_28.bw2package")

# The water use method needs to be updated due to the fact that it was wrongly created.
# Filter the methods list to find the desired method for 'IMPACTWorld+ (Default_Recommended_Midpoint 1.28)'.
# Then, load the method and its characterization factors into the WS_lst variable.
IWP_mid = [m for m in methods if 'IMPACTWorld+ (Default_Recommended_Midpoint 1.28)' in m[0]]
WS = Method(IWP_mid[9])
WS_lst = WS.load()

# Define and remove certain characterization factors (CF) from the WS_lst.
ex_CF = (('biosphere3', '2404b41a-2eed-4e9d-8ab6-783946fdf5d6'), -42.95353086694035)
ex_CF_ocean = (('biosphere3', '4f0f15b3-b227-4cdc-b0b3-6412d55695d5'), 0)
WS_lst.remove(ex_CF)
WS_lst.remove(ex_CF_ocean)

# Define a new characterization factor and add it to the WS_lst.
new_CF = (('biosphere3', '8c1494a5-4987-4715-aa2d-1908c495f4eb'), 42.95353086694035)
WS_lst.append(new_CF)

# Create a new Method instance for the updated 'Water scarcity' method.
# Register the new method and write the updated characterization factors to it.
new_WS = Method(('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)', 'Water scarcity'))
new_WS.register()
new_WS.write(WS_lst)

Assigning methods and units required for axis labels in visualisations

In [ ]:
IWP_mid = [m for m in methods if 'IMPACTWorld+ (Default_Recommended_Midpoint 1.28)' in m[0]]
units_IWP_mid=['kg CO2eq.','kg CO2eq.','MJ dep.','kg dep.', 'kg NMVOCeq.','kg CFC-11eq.','CTUe','CTUh','CTUh','m3eq.','kg SO2eq.','kg SO2eq.','kg PO4eq.','kg Neq.', 'm2eq.', 'm2eq*yr.', 'kg PM2.5eq.', 'Bq C14eq.']

In [ ]:
IWP_end = [m for m in methods if 'IMPACTWorld+ (Default_Recommended_Damage 1.46)' in m[0]]

## Selection of processes for the configurations

Defining the 'use of aircraft' processes for all configurations, SSP scenarios and fuels from the imported LCI databases

In [ ]:
#Aircraft use processes for the typical mission (200 nmi)

#KERSOSENE FUELED AIRCRAFT
#conventional 
typ_aircraft_conv_kero_NDC = Database('GENESIS_2030_conventional_NDC').get('F2FEAAE907F648A69BFB501E91366D35')
typ_aircraft_conv_kero_Base = Database("GENESIS_2030_conventional_Base").get('F2FEAAE907F648A69BFB501E91366D35')
typ_aircraft_conv_kero_PkBudg500 = Database("GENESIS_2030_conventional_PkBudg500").get('F2FEAAE907F648A69BFB501E91366D35')

#Gt-bat 
typ_aircraft_GT_bat_kero_NDC = Database("GENESIS_2030_GT-bat_NDC").get('6F97863F6BEF4DE383907C08DE912E6D')
typ_aircraft_GT_bat_kero_Base = Database("GENESIS_2030_GT-bat_Base").get('6F97863F6BEF4DE383907C08DE912E6D')
typ_aircraft_GT_bat_kero_PkBudg500 = Database("GENESIS_2030_GT-bat_PkBudg500").get('6F97863F6BEF4DE383907C08DE912E6D')


#AAF FUELED AIRCRAFT
#Conventional
typ_aircraft_conv_AAF_NDC = Database("GENESIS_2030_conventional_NDC").get('BE40C283116D46E986C246F2B77A86D7')
typ_aircraft_conv_AAF_Base = Database("GENESIS_2030_conventional_Base").get('BE40C283116D46E986C246F2B77A86D7')
typ_aircraft_conv_AAF_PkBudg500 = Database("GENESIS_2030_conventional_PkBudg500").get('BE40C283116D46E986C246F2B77A86D7')
typ_aircraft_conv_ILUC_NDC = Database("GENESIS_2030_conventional_NDC_ILUC").get('BE40C283116D46E986C246F2B77A86D7')

#Gt-bat
typ_aircraft_GT_bat_AAF_NDC = Database("GENESIS_2030_GT-bat_NDC").get('DFDABF367DB344E4AFEF072D2A7634F3')
typ_aircraft_GT_bat_AAF_Base = Database("GENESIS_2030_GT-bat_Base").get('DFDABF367DB344E4AFEF072D2A7634F3')
typ_aircraft_GT_bat_AAF_PkBudg500 = Database("GENESIS_2030_GT-bat_PkBudg500").get('DFDABF367DB344E4AFEF072D2A7634F3'
typ_aircraft_GT_bat_ILUC_NDC = Database("GENESIS_2030_GT-bat_NDC_ILUC").get('DFDABF367DB344E4AFEF072D2A7634F3')


In [ ]:
#Aircraft use processes for the design mission (600 nmi)

#KERSOSENE FUELED AIRCRAFT
#conventional
des_aircraft_conv_kero_NDC = Database('GENESIS_2030_conventional_NDC').get('0808F060E0F94D1BB0380DFA4A9065F5')
des_aircraft_conv_kero_Base = Database("GENESIS_2030_conventional_Base").get('0808F060E0F94D1BB0380DFA4A9065F5')
des_aircraft_conv_kero_PkBudg500 = Database("GENESIS_2030_conventional_PkBudg500").get('0808F060E0F94D1BB0380DFA4A9065F5')

#Gt-bat
des_aircraft_GT_bat_kero_NDC = Database("GENESIS_2030_GT-bat_NDC").get('3FCE85DD88EF4504A6F7D6326303F33D')
des_aircraft_GT_bat_kero_Base = Database("GENESIS_2030_GT-bat_Base").get('3FCE85DD88EF4504A6F7D6326303F33D')
des_aircraft_GT_bat_kero_PkBudg500 = Database("GENESIS_2030_GT-bat_PkBudg500").get('3FCE85DD88EF4504A6F7D6326303F33D')

#AAF FUELED AIRCRAFT
#Conventional
des_aircraft_conv_AAF_NDC = Database("GENESIS_2030_conventional_NDC").get('BA3B99FBFBDA4677867DD3202F1BD7CC')
des_aircraft_conv_AAF_Base = Database("GENESIS_2030_conventional_Base").get('BA3B99FBFBDA4677867DD3202F1BD7CC')
des_aircraft_conv_AAF_PkBudg500 = Database("GENESIS_2030_conventional_PkBudg500").get('BA3B99FBFBDA4677867DD3202F1BD7CC')
des_aircraft_conv_ILUC_NDC = Database("GENESIS_2030_conventional_NDC_ILUC").get('BA3B99FBFBDA4677867DD3202F1BD7CC')

#Gt-bat
des_aircraft_GT_bat_AAF_NDC = Database("GENESIS_2030_GT-bat_NDC").get('8C441EC088644DCAA1FF915F3E9AF97F')
des_aircraft_GT_bat_AAF_Base = Database("GENESIS_2030_GT-bat_Base").get('8C441EC088644DCAA1FF915F3E9AF97F')
des_aircraft_GT_bat_AAF_PkBudg500 = Database("GENESIS_2030_GT-bat_PkBudg500").get('8C441EC088644DCAA1FF915F3E9AF97F')
des_aircraft_GT_bat_ILUC_NDC = Database("GENESIS_2030_GT-bat_NDC_ILUC").get('8C441EC088644DCAA1FF915F3E9AF97F')



## Monte Carlo Calculation (500 runs, seed=35)

Definition of function for Monte Carlo simulations, with a seed number of 35. The code in the next cell is based on the work of Chris Mutel and Pascal Lesage https://github.com/PoutineAndRosti/Brightway-Seminar-2017/blob/master/Day%201%20PM/Brightway%20and%20uncertainty.ipynb

In [ ]:
def multiImpactMonteCarloLCA(functional_unit, list_methods, iterations):
    
    # Step 1 Create a MonteCarloLCA object with some functional unit but no method, set seed
    MC_lca = MonteCarloLCA(functional_unit, seed=35)
    #run .lci to build the A and B matrices 
    MC_lca.lci()
    
    # Step 2 Create an empty 'C_matrices' dictionary, that will collect characterization factor matrices (C matrices). 
    #Keys will be the tuple representing the method name, and the values will be the actual matrices.
    C_matrices = {}
    
    # Step 3 for loop that iterates over a list of method names and stores the method_name:C_matrix in the dictionary
    for method in list_methods:
        MC_lca.switch_method(method)
        #add the C matrix for the method in question to the C_matrices dictionary
        C_matrices[method] = MC_lca.characterization_matrix 

    # Step 4 empty array (np.empty) of dimension equal to the number of methods that will be considered (rows) by the number of iterations required (columns).
    results = np.empty((len(list_methods), iterations)) 
    
    # Step 5 Populating the array using a for loop over the number of MonteCarlo iterations required (use next to rebuild the A and B matrices). 
    for iteration in range(iterations):
        next(MC_lca)
        for method_index, method in enumerate(list_methods):
            results[method_index, iteration] = (C_matrices[method]*MC_lca.inventory).sum()
    return results

Setting up ImpactWorld+ for midpoint LCIA method 

In [ ]:
IWP_mid = [m for m in methods if 'IMPACTWorld+ (Default_Recommended_Midpoint 1.28)' in m[0]]
units_IWP_mid=['[kg CO2eq.]','[kg CO2eq.]','[MJ dep.]','[kg dep.]', '[kg NMVOCeq.]','[kg CFC-11eq.]','[CTUe]','[CTUh]','[CTUh]','[m3eq.]','[kg SO2eq.]','[kg SO2eq.]','[kg PO4eq.]','[kg Neq.]', '[m2eq.]', '[m2eq*yr.]', '[kg PM2.5eq.]', '[Bq C14eq.]']

Setting up ImpactWorld+ for endpoint LCIA method 

In [ ]:
IWP_end = [m for m in methods if 'IMPACTWorld+ (Default_Recommended_Damage 1.46)' in m[0]]

### Design mission

#### Endpoint

Running the Monte Carlo simulations for the design mission configurations defined above (endpoint). Each configuration takes around 2.5 hours.

In [ ]:
r_des_aircraft_conv_kero_NDC=multiImpactMonteCarloLCA({des_aircraft_conv_kero_NDC:1}, IWP_end, 500) 
r_des_aircraft_conv_kero_Base= multiImpactMonteCarloLCA({des_aircraft_conv_kero_Base:1}, IWP_end, 500) 
r_des_aircraft_conv_kero_PkBudg500=multiImpactMonteCarloLCA({des_aircraft_conv_kero_PkBudg500:1}, IWP_end, 500) 
r_des_aircraft_GT_bat_kero_NDC=multiImpactMonteCarloLCA({des_aircraft_GT_bat_kero_NDC:1}, IWP_end, 500) 
r_des_aircraft_GT_bat_kero_Base= multiImpactMonteCarloLCA({des_aircraft_GT_bat_kero_Base:1}, IWP_end, 500) 
r_des_aircraft_GT_bat_kero_PkBudg500=multiImpactMonteCarloLCA({des_aircraft_GT_bat_kero_PkBudg500:1}, IWP_end, 500)  
r_des_aircraft_conv_AAF_NDC = multiImpactMonteCarloLCA({des_aircraft_conv_AAF_NDC:1}, IWP_end, 500) 
r_des_aircraft_conv_AAF_Base = multiImpactMonteCarloLCA({des_aircraft_conv_AAF_Base:1}, IWP_end, 500) 
r_des_aircraft_conv_AAF_PkBudg500 = multiImpactMonteCarloLCA({des_aircraft_conv_AAF_PkBudg500:1}, IWP_end, 500) 
r_des_aircraft_GT_bat_AAF_NDC =multiImpactMonteCarloLCA({des_aircraft_GT_bat_AAF_NDC:1}, IWP_end, 500) 
r_des_aircraft_GT_bat_AAF_Base =multiImpactMonteCarloLCA({des_aircraft_GT_bat_AAF_Base:1}, IWP_end, 500) 
r_des_aircraft_GT_bat_AAF_PkBudg500 =multiImpactMonteCarloLCA({des_aircraft_GT_bat_AAF_PkBudg500:1}, IWP_end, 500)
r_des_aircraft_conv_ILUC_NDC = multiImpactMonteCarloLCA({des_aircraft_conv_ILUC_NDC:1}, IWP_end, 500) 
r_des_aircraft_GT_bat_ILUC_NDC =multiImpactMonteCarloLCA({des_aircraft_GT_bat_ILUC_NDC:1}, IWP_end, 500) 

#saving the results as a pickle
var_endpoint = [r_des_aircraft_conv_kero_NDC,r_des_aircraft_conv_kero_Base,r_des_aircraft_conv_kero_PkBudg500,r_des_aircraft_GT_bat_kero_NDC, r_des_aircraft_GT_bat_kero_Base,r_des_aircraft_GT_bat_kero_PkBudg500,r_des_aircraft_conv_AAF_NDC,r_des_aircraft_conv_AAF_Base,r_des_aircraft_conv_AAF_PkBudg500,r_des_aircraft_GT_bat_AAF_NDC,r_des_aircraft_GT_bat_AAF_Base,r_des_aircraft_GT_bat_AAF_PkBudg500,r_des_aircraft_conv_ILUC_NDC,r_des_aircraft_GT_bat_ILUC_NDC] 
with open(f"MC_design_endpoint_2030.pkl", "wb+") as f:
    pickle.dump(var_endpoint, f)


#### Midpoint

Running the Monte Carlo simulations for the design mission configurations defined above (midpoint). Each configuration takes around 2.5 hours.

In [ ]:
r_des_aircraft_conv_kero_NDC=multiImpactMonteCarloLCA({des_aircraft_conv_kero_NDC:1}, IWP_mid, 500) 
r_des_aircraft_conv_kero_Base= multiImpactMonteCarloLCA({des_aircraft_conv_kero_Base:1}, IWP_mid, 500) 
r_des_aircraft_conv_kero_PkBudg500=multiImpactMonteCarloLCA({des_aircraft_conv_kero_PkBudg500:1}, IWP_mid, 500) 
r_des_aircraft_GT_bat_kero_NDC=multiImpactMonteCarloLCA({des_aircraft_GT_bat_kero_NDC:1}, IWP_mid, 500) 
r_des_aircraft_GT_bat_kero_Base= multiImpactMonteCarloLCA({des_aircraft_GT_bat_kero_Base:1}, IWP_mid, 500) 
r_des_aircraft_GT_bat_kero_PkBudg500=multiImpactMonteCarloLCA({des_aircraft_GT_bat_kero_PkBudg500:1}, IWP_mid, 500)  
r_des_aircraft_conv_AAF_NDC = multiImpactMonteCarloLCA({des_aircraft_conv_AAF_NDC:1}, IWP_mid, 500) 
r_des_aircraft_conv_AAF_Base = multiImpactMonteCarloLCA({des_aircraft_conv_AAF_Base:1}, IWP_mid, 500) 
r_des_aircraft_conv_AAF_PkBudg500 = multiImpactMonteCarloLCA({des_aircraft_conv_AAF_PkBudg500:1}, IWP_mid, 500) 
r_des_aircraft_GT_bat_AAF_NDC =multiImpactMonteCarloLCA({des_aircraft_GT_bat_AAF_NDC:1}, IWP_mid, 500) 
r_des_aircraft_GT_bat_AAF_Base =multiImpactMonteCarloLCA({des_aircraft_GT_bat_AAF_Base:1}, IWP_mid, 500) 
r_des_aircraft_GT_bat_AAF_PkBudg500 =multiImpactMonteCarloLCA({des_aircraft_GT_bat_AAF_PkBudg500:1}, IWP_mid, 500)
r_des_aircraft_conv_ILUC_NDC = multiImpactMonteCarloLCA({des_aircraft_conv_ILUC_NDC:1}, IWP_mid, 500) 
r_des_aircraft_GT_bat_ILUC_NDC =multiImpactMonteCarloLCA({des_aircraft_GT_bat_ILUC_NDC:1}, IWP_mid, 500) 

var_midpoint = [r_des_aircraft_conv_kero_NDC,r_des_aircraft_conv_kero_Base,r_des_aircraft_conv_kero_PkBudg500,r_des_aircraft_GT_bat_kero_NDC, r_des_aircraft_GT_bat_kero_Base,r_des_aircraft_GT_bat_kero_PkBudg500,r_des_aircraft_conv_AAF_NDC,r_des_aircraft_conv_AAF_Base,r_des_aircraft_conv_AAF_PkBudg500,r_des_aircraft_GT_bat_AAF_NDC,r_des_aircraft_GT_bat_AAF_Base,r_des_aircraft_GT_bat_AAF_PkBudg500,r_des_aircraft_conv_ILUC_NDC,r_des_aircraft_GT_bat_ILUC_NDC]
with open(f"MC_design_midpoint_2030.pkl", "wb+") as f:
    pickle.dump(var_midpoint, f)

### Typical mission

#### Endpoint

Running the Monte Carlo simulations for the typical mission configurations (endpoint). Each configuration takes around 2.5 hours.

In [ ]:
r_typ_aircraft_conv_kero_NDC=multiImpactMonteCarloLCA({typ_aircraft_conv_kero_NDC:1}, IWP_end, 500) 
r_typ_aircraft_conv_kero_Base= multiImpactMonteCarloLCA({typ_aircraft_conv_kero_Base:1}, IWP_end, 500) 
r_typ_aircraft_conv_kero_PkBudg500=multiImpactMonteCarloLCA({typ_aircraft_conv_kero_PkBudg500:1}, IWP_end, 500) 
r_typ_aircraft_GT_bat_kero_NDC=multiImpactMonteCarloLCA({typ_aircraft_GT_bat_kero_NDC:1}, IWP_end, 500) 
r_typ_aircraft_GT_bat_kero_Base= multiImpactMonteCarloLCA({typ_aircraft_GT_bat_kero_Base:1}, IWP_end, 500) 
r_typ_aircraft_GT_bat_kero_PkBudg500=multiImpactMonteCarloLCA({typ_aircraft_GT_bat_kero_PkBudg500:1}, IWP_end, 500)  
r_typ_aircraft_conv_AAF_NDC = multiImpactMonteCarloLCA({typ_aircraft_conv_AAF_NDC:1}, IWP_end, 500) 
r_typ_aircraft_conv_AAF_Base = multiImpactMonteCarloLCA({typ_aircraft_conv_AAF_Base:1}, IWP_end, 500) 
r_typ_aircraft_conv_AAF_PkBudg500 = multiImpactMonteCarloLCA({typ_aircraft_conv_AAF_PkBudg500:1}, IWP_end, 500) 
r_typ_aircraft_GT_bat_AAF_NDC =multiImpactMonteCarloLCA({typ_aircraft_GT_bat_AAF_NDC:1}, IWP_end, 500) 
r_typ_aircraft_GT_bat_AAF_Base =multiImpactMonteCarloLCA({typ_aircraft_GT_bat_AAF_Base:1}, IWP_end, 500) 
r_typ_aircraft_GT_bat_AAF_PkBudg500 =multiImpactMonteCarloLCA({typ_aircraft_GT_bat_AAF_PkBudg500:1}, IWP_end, 500)
r_typ_aircraft_conv_ILUC_NDC = multiImpactMonteCarloLCA({typ_aircraft_conv_ILUC_NDC:1}, IWP_end, 500) 
r_typ_aircraft_GT_bat_ILUC_NDC =multiImpactMonteCarloLCA({typ_aircraft_GT_bat_ILUC_NDC:1}, IWP_end, 500) 

var_endpoint = [r_typ_aircraft_conv_kero_NDC,r_typ_aircraft_conv_kero_Base,r_typ_aircraft_conv_kero_PkBudg500,r_typ_aircraft_GT_bat_kero_NDC, r_typ_aircraft_GT_bat_kero_Base,r_typ_aircraft_GT_bat_kero_PkBudg500,r_typ_aircraft_conv_AAF_NDC,r_typ_aircraft_conv_AAF_Base,r_typ_aircraft_conv_AAF_PkBudg500,r_typ_aircraft_GT_bat_AAF_NDC,r_typ_aircraft_GT_bat_AAF_Base,r_typ_aircraft_GT_bat_AAF_PkBudg500,r_typ_aircraft_conv_ILUC_NDC,r_typ_aircraft_GT_bat_ILUC_NDC]
with open(f"MC_typical_endpoint_2030.pkl", "wb+") as f:
    pickle.dump(var_endpoint, f)


#### Midpoint

Running the Monte Carlo simulations for the typical mission configurations (midpoint). Each configuration takes around 2.5 hours.

In [ ]:
r_typ_aircraft_conv_kero_NDC=multiImpactMonteCarloLCA({typ_aircraft_conv_kero_NDC:1}, IWP_mid, 500) 
r_typ_aircraft_conv_kero_Base= multiImpactMonteCarloLCA({typ_aircraft_conv_kero_Base:1}, IWP_mid, 500) 
r_typ_aircraft_conv_kero_PkBudg500=multiImpactMonteCarloLCA({typ_aircraft_conv_kero_PkBudg500:1}, IWP_mid, 500) 
r_typ_aircraft_GT_bat_kero_NDC=multiImpactMonteCarloLCA({typ_aircraft_GT_bat_kero_NDC:1}, IWP_mid, 500) 
r_typ_aircraft_GT_bat_kero_Base= multiImpactMonteCarloLCA({typ_aircraft_GT_bat_kero_Base:1}, IWP_mid, 500) 
r_typ_aircraft_GT_bat_kero_PkBudg500=multiImpactMonteCarloLCA({typ_aircraft_GT_bat_kero_PkBudg500:1}, IWP_mid, 500)  
r_typ_aircraft_conv_AAF_NDC = multiImpactMonteCarloLCA({typ_aircraft_conv_AAF_NDC:1}, IWP_mid, 500) 
r_typ_aircraft_conv_AAF_Base = multiImpactMonteCarloLCA({typ_aircraft_conv_AAF_Base:1}, IWP_mid, 500) 
r_typ_aircraft_conv_AAF_PkBudg500 = multiImpactMonteCarloLCA({typ_aircraft_conv_AAF_PkBudg500:1}, IWP_mid, 500) 
r_typ_aircraft_GT_bat_AAF_NDC =multiImpactMonteCarloLCA({typ_aircraft_GT_bat_AAF_NDC:1}, IWP_mid, 500) 
r_typ_aircraft_GT_bat_AAF_Base =multiImpactMonteCarloLCA({typ_aircraft_GT_bat_AAF_Base:1}, IWP_mid, 500) 
r_typ_aircraft_GT_bat_AAF_PkBudg500 =multiImpactMonteCarloLCA({typ_aircraft_GT_bat_AAF_PkBudg500:1}, IWP_mid, 500)
r_typ_aircraft_conv_ILUC_NDC = multiImpactMonteCarloLCA({typ_aircraft_conv_ILUC_NDC:1}, IWP_mid, 500) 
r_typ_aircraft_GT_bat_ILUC_NDC =multiImpactMonteCarloLCA({typ_aircraft_GT_bat_ILUC_NDC:1}, IWP_mid, 500) 

var_midpoint = [r_typ_aircraft_conv_kero_NDC,r_typ_aircraft_conv_kero_Base,r_typ_aircraft_conv_kero_PkBudg500,r_typ_aircraft_GT_bat_kero_NDC, r_typ_aircraft_GT_bat_kero_Base,r_typ_aircraft_GT_bat_kero_PkBudg500,r_typ_aircraft_conv_AAF_NDC,r_typ_aircraft_conv_AAF_Base,r_typ_aircraft_conv_AAF_PkBudg500,r_typ_aircraft_GT_bat_AAF_NDC,r_typ_aircraft_GT_bat_AAF_Base,r_typ_aircraft_GT_bat_AAF_PkBudg500,r_typ_aircraft_conv_ILUC_NDC,r_typ_aircraft_GT_bat_ILUC_NDC]
with open(f"MC_typical_midpoint_2030.pkl", "wb+") as f:
    pickle.dump(var_midpoint, f)
